## Python Week 7

November 9, 2019

If the installation was done properly on October 26, we are ready to start machine learning projects.
In the following weeks, we'll make a team and pick a project that you are interested in. In order to give you better ideas and understand what AI can do, we'll go over a few examples drawn from engineering and science. 

Today, let's start with number recognition and image denoising using autoencoder.
We'll use the dataset that we previously played with for our plotting. So the data should be in your cloned repository.


In [ ]:
import numpy as np
import tensorflow 

if tensorflow.__version__ < '2.0.0':
    import keras 
    from keras.models import Model
    from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
    
elif tensorflow.__version__ >= '2.0.0':
    from tensorflow import keras
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout

## keras functions are well documented in the Keras Documentaion 
## https://keras.io
print("using tensorflow ", tensorflow.__version__)

In [ ]:
Xtest3D = np.load('xtest.pickle', allow_pickle=True)
Ytest = np.load('ytest.pickle', allow_pickle=True)

In [ ]:
#Xtrain3D = np.load('xtrain.pickle', allow_pickle=True)
#Ytrain = np.load('ytrain.pickle', allow_pickle=True)

In [ ]:
## original images have unsigned integer from 0 to 255 (8bit representation of gray scale color) 
## Since the convolutions are numerical computation, we change the data type to float32 and then 
## perform normalizations to [0,1] range

Xtest3D = Xtest3D.astype('float32')/255.
#Xtrain3D = Xtrain3D.astype('float32')/255.

In [ ]:
## in order to conform to the shape of inputs to Conv2D
## Conv2D expects 4D array
## 1st index - sample index
## 2nd index - image x
## 3rd index - image y
## 4th index - channel (Ex. RGB values in color images)

Xtest = Xtest3D.reshape((*Xtest3D.shape, 1))
#Xtrain = Xtrain3D.reshape((*Xtrain3D.shape, 1))

In [ ]:
## Since the computing power and time is limitted, let's reduce the size of training and test datasets 

Xtrain = Xtest[0:600,:]
Ytrain = Ytest[0:600]

Xtest = Xtest[1000:1100,:]
Ytest = Ytest[1000:1100]


In [ ]:
## Let's check the shape and type of the our test dataset

print(Xtest.shape, Ytest.shape)
print(Xtest.dtype, Ytest.dtype)

In [ ]:
print(Xtrain.shape, Ytrain.shape)
print(Xtrain.dtype, Ytrain.dtype)

In [ ]:
## Let's define our neural network model that consists of Convolutional, Flattening, MaxPooling and UpSampling layers
## We can go deep into each layers and optimization if necessary and if our members are interested.

def model(choice = 2):
    '''
    input parameters
      choice - 1, classification of mnist handwriting images
               2, encoding and decoding for denoising
               
    output
      keras model defining the network from an input to the final output
    '''
    
    
    input_img = Input(shape=(28, 28, 1))
    x1 = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
    x2 = MaxPooling2D((2,2), padding='same')(x1)
    x3 = Conv2D(32, (3,3), activation='relu', padding='same')(x2)
    
    encoded = MaxPooling2D((2,2), padding='same')(x3)
    c1 = Flatten()(encoded)
    c2 = Dense(128, activation='relu')(c1)
    c3 = Dropout(0.2)(c2)
    
    ## this is the final output for classification
    classifierOutput = Dense(10, activation='softmax')(c3)
    
    ## The following layers from y4 to decoded is for denoising
    ## we'll cover this in the next meeting
    y4 = Conv2D(32, (3,3), activation='relu', padding='same')(encoded)
    y3 = UpSampling2D((2,2))(y4)
    y2 = Conv2D(32, (3,3), activation='relu', padding='same')(y3)
    y1 = UpSampling2D((2,2))(y2)
    
    decoded = Conv2D(1, (3,3), activation='sigmoid', padding='same')(y1)
    
    if choice == 1:
        return Model(input_img, classifierOutput)
    else: 
        
        ## This part is for the next meeting
        
        return Model(input_img, decoded)    

In [ ]:
## Here we call a function and get our model defintion.
myclassifier = model(1)

In [ ]:
## Once a model is defined, we need to configure the model for trainging 
## by selecting optimizer and loss function

myclassifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## This is the traing stage that is compute intensive and may require high performance computers
## or GPU machines if the size of the training dataset is huge
## 
## In the begining of notebook, we decimated the size of the training dataset to finish our meeting on time
## If you increase the epochs, the number of data repetition,
## the running time will increase linearly proportional to the epochs.
myclassifier.fit(Xtrain, Ytrain, epochs=20, batch_size=128)

In [ ]:
Xpred = myclassifier.predict(Xtest)

In [ ]:
myclassifier.evaluate(Xtest, Ytest)

In [ ]:
# In the above, the test accuracy is 94%
# Our model made wrong predictions in 6 cases out of 100.

In [ ]:
## Now, let's find out which image (handwriting) is not understood(classified) correctly.
XpredFinal = [np.argmax(x) for x in Xpred]

In [ ]:
XpredFinalArray = np.array(XpredFinal, dtype='float32')
YtestFinalArray = Ytest.astype('float32')

In [ ]:
Errors = Ytest - XpredFinalArray

In [ ]:
maxErrorLocation = np.argmax(np.abs(Errors))

In [ ]:
ximage = Xtest[maxErrorLocation,  :]

In [ ]:
ximage2d = ximage.reshape(ximage.shape[0], ximage.shape[1])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(ximage2d)

# this figure looks 4 but at the same time, it is close to 9.
# our model predition is 9. But it is not totally wrong, I think.


In [ ]:
# prediction 
print(XpredFinalArray[maxErrorLocation])

In [ ]:
# Correction Answer
print(Ytest[maxErrorLocation])